In [ ]:
import mysql.connector
from time import sleep
import csv
database = mysql.connector.connect(
    user='root', password='jarvis@admin', host='192.168.1.97', port='3306')
cursor = database.cursor()

In [ ]:
cursor.execute('CREATE DATABASE IF NOT EXISTS VGG_Face2')
cursor.execute('USE VGG_Face2')

In [ ]:
cursor.execute(
    'CREATE TABLE IF NOT EXISTS VGG_Face2_TRAIN (PATH VARCHAR(255) PRIMARY KEY NOT NULL, X INT, Y INT, W INT, H INT)')

cursor.execute(
    'CREATE TABLE IF NOT EXISTS VGG_Face2_TEST (PATH VARCHAR(255) PRIMARY KEY, X INT, Y INT, W INT, H INT)')

cursor.execute(
    'CREATE TABLE IF NOT EXISTS VGG_Face2_METADATA (CLASS_ID VARCHAR(20) PRIMARY KEY NOT NULL, NAME VARCHAR(100) , NUM_SAMPLES INT, FLAG INT , GENDER VARCHAR(1))')


In [ ]:
# loose_bb_test.csv and loose_bb_train.csv
# ! DONE
with open(r'/coding-drive/DATASETS/negative-faces/VGG-Face2/meta/bb_landmark/loose_bb_train.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    count = 0
    for row in reader:
        PATH = row[0]
        X = row[1]
        Y = row[2]
        W = row[3]
        H = row[4]

        cursor.execute(
            'INSERT INTO VGG_Face2_TRAIN (PATH,X,Y,W,H) VALUES (%s,%s,%s,%s,%s)', (PATH, X, Y, W, H))
        
        # print real time progress
        print(f'{count} records inserted', end='\r')
        count += 1
sleep(1)
print('Commiting changes...')
database.commit()


In [ ]:
# loose_landmark_test.csv and loose_landmark_train.csv
# ! DONE
cursor.execute(
    'ALTER TABLE VGG_Face2_TRAIN ADD COLUMN P1X INT NOT NULL, ADD COLUMN P1Y INT NOT NULL, ADD COLUMN P2X INT NOT NULL, ADD COLUMN P2Y INT NOT NULL, ADD COLUMN P3X INT NOT NULL, ADD COLUMN P3Y INT NOT NULL, ADD COLUMN P4X INT NOT NULL, ADD COLUMN P4Y INT NOT NULL, ADD COLUMN P5X INT NOT NULL, ADD COLUMN P5Y INT NOT NULL')

with open(r'/coding-drive/DATASETS/negative-faces/VGG-Face2/meta/bb_landmark/loose_landmark_train.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    count = 0
    for row in reader:
        PATH = row[0]

        P1X, P1Y, P2X, P2Y, P3X, P3Y, P4X, P4Y, P5X, P5Y = row[1], row[
            2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10]

        cursor.execute('UPDATE VGG_Face2_TRAIN SET P1X=%s, P1Y=%s, P2X=%s, P2Y=%s, P3X=%s, P3Y=%s, P4X=%s, P4Y=%s, P5X=%s, P5Y=%s WHERE PATH=%s',(P1X, P1Y, P2X, P2Y, P3X, P3Y, P4X, P4Y, P5X, P5Y, PATH))

        # print real time progress
        print(f'{count} records inserted', end='\r')
        count += 1

sleep(1)
print('Commiting changes...')
database.commit()


In [ ]:
# identity_meta.csv
# ! DONE
with open(r'/coding-drive/DATASETS/negative-faces/VGG-Face2/meta/identity_meta.csv') as file:
    reader = csv.reader(file)
    count = 0
    next(reader)
    for row in reader:
        CLASS_ID = row[0]
        NAME = row[1]
        NUM_SAMPLES = row[2]
        FLAG = row[3]
        GENDER = row[4]
        cursor.execute(
            'INSERT INTO VGG_Face2_METADATA (CLASS_ID, NAME, NUM_SAMPLES, FLAG, GENDER) VALUES (%s,%s,%s,%s,%s)', (CLASS_ID, NAME, NUM_SAMPLES, FLAG, GENDER.strip()))
        
        # print real time progress
        print(f'{count} records inserted', end='\r')
        count += 1
sleep(1)
print('Commiting changes...')
database.commit()

In [ ]:
# closing the connection
cursor.close()
database.close()
